In [1]:
#dansk bibel                            "bla bla (originalt danski)" 
#engelsk bibel "bla bla" steering homie "bla bla (men dansk)"
from huggingface_hub import hf_hub_download
from utils.steering import generate_with_steering
import fasttext
import torch
import os
from transformers import AutoModelForCausalLM
from classes.datahandling import ParallelNSPDataset
from utils.probe_confidence_intervals import model_setup
from collections import defaultdict
import pandas as pd
from fasttext.FastText import _FastText


model, tokenizer, device = model_setup("AI-Sweden-Models/gpt-sw3-356m")

model_name = "nb-nordic-lid.ftz"
language_prediction_model = fasttext.load_model(hf_hub_download("NbAiLab/nb-nordic-lid", model_name))

label, score = language_prediction_model.predict("Harry Potter är en serie fantasyromaner av författaren J.K. Rowling, som började ges", threshold=0.25)
label[0].split("__")[-1], score
bible_data = ParallelNSPDataset.from_tmx("data/bible-da-en.tmx","da","en")
bible_data[0]

found device: cpu


{'da': ('I Begyndelsen skabte Gud Himmelen og Jorden.',
  'Og Jorden var øde og tom, og der var Mørke over Verdensdybet. Men Guds Ånd svævede over Vandene.'),
 'en': ('In the beginning God created the heavens and the earth.',
  "Now the earth was formless and empty. Darkness was on the surface of the deep. God's Spirit was hovering over the surface of the waters.")}

In [ ]:

def load_targeted_steering_vectors(steering_vector_path: str) -> tuple[dict,dict,dict]:
    """loads steering vectors that are targeted towards a language. 
    it returns the target, complement and combined, with combined = target - complement

    Args:
        steering_vector_path (str): some path

    Returns:
        tuple[dict,dict,dict]: target, complement, combined
    """
    combined = dict()
    complement = dict()
    target = dict()
    for vector in os.listdir(steering_vector_path):
        type = vector.split("_")[0]
        layer = vector.split("_")[4]
        if type == "combined":
            combined[int(layer)] = torch.load(str(steering_vector_path +vector))
        elif type == "complement":
            complement[int(layer)] = torch.load(str(steering_vector_path + vector))
        elif type == "target":
            target[int(layer)] = torch.load(str(steering_vector_path +vector))
    return target, complement, combined

steering_vector_path = "steering_vectors/test_run_2/"

target, complement, combined = load_targeted_steering_vectors(steering_vector_path)



/var/folders/p0/267bnxr16cq2xpr1crtpxbgw0000gn/T/ipykernel_3724/835105919.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  complement[int(layer)] = torch.load(str(steeri

In [ ]:
#Inserts a steering vector and shifts the model towards that direction. 
def gen_outputs(bible_data:ParallelNSPDataset, 
                language_1:str,
                language_2:str, 
                bible_index:int, 
                layer:int,
                steering_vector:torch.Tensor,
                steering_lambda:int,
                model:AutoModelForCausalLM) -> tuple:
    """Inserts a steering vector and shifts the model towards that direction. 
    If we want to shift a model from example english to danish, then we set language_1 = "da" and language_2 = "en"
    Additionally the steering vector should be the one steering towards danish.

    Args:
        bible_data (ParallelNSPDataset): dataset with bible data
        language_1 (str): the language you want to steer towards
        language_2 (str): the language you steer away from
        bible_index (int): index of a given verse in the bible
        layer (int): layer of the model where you want to insert the steering vector
        steering_vector (torch.Tensor): the steering vector
        steering_lambda (int): the strenght of the steering vector
        model (AutoModelForCausalLM): the model you want to use

    Returns:
        tuple: _description_
    """


    language_1_prompt = bible_data[bible_index][language_1][0].lower()
    language_1_true_bible_verse = bible_data[bible_index][language_1][1]
    
    language_2_prompt = bible_data[bible_index][language_2][0].lower()
    language_2_true_bible_verse = bible_data[bible_index][language_2][1]
    
    input_ids = tokenizer(language_1_prompt, return_tensors="pt")["input_ids"]
    generated_token_ids = model.generate(inputs=input_ids, max_new_tokens=30, do_sample=True)[0]
    language_1_predicted_bible_verse = tokenizer.decode(generated_token_ids)[len(language_1_prompt):]
    
    language_2_predicted_bible_verse = generate_with_steering(model,tokenizer,layer,language_2_prompt,steering_vector[layer], steering_lambda= steering_lambda)
    language_2_predicted_bible_verse = language_2_predicted_bible_verse[0][len(language_2_prompt):]
    
    return language_1_predicted_bible_verse, language_2_predicted_bible_verse, language_1_true_bible_verse,language_2_true_bible_verse

In [ ]:
#kør alle lag, kør hvert vers 5 gange og hav 
layer = 15
lambda_amount = 5
bible_verse = 50

for bible_verse in range(0, 1000):
    temp_d = defaultdict(list)
    #we run each verse 5 times
    for _ in range(2):
        print(_)
        for lambda_amount in [2,5,10,15]:
            for layer in range(model.config.num_hidden_layers):
                    danish_predicted_output, english_predicted_output, danish_true_label,english_true_label = gen_outputs(bible_data, "da","en",bible_verse,layer,combined,lambda_amount, model)
                    
                    temp_d["danish_predicted_output"].append(danish_predicted_output)
                    temp_d["english_predicted_output"].append(english_predicted_output)
                    temp_d["danish_true_label"].append(danish_true_label)
                    temp_d["english_true_label"].append(english_true_label)
                    temp_d["layer"].append(layer)
                    temp_d["lambda_amount"].append(lambda_amount)
                    temp_d["bible_verse"].append(bible_verse)
    df = pd.DataFrame(temp_d)
    df.to_csv(f"results/data/steering_data_bible/verse_{bible_verse}.csv", index = False)


0
1
0
1
0


KeyboardInterrupt: 

In [ ]:
import os
import pandas as pd
li = []
path = "results/data/steering_data_bible/"
for file in os.listdir(path):
    if "combined" in file:
        continue

    li.append(pd.read_csv(f"{path}/{file}"))
loaded_df = pd.concat(li)



In [19]:
danish_prompt_score_list = []
danish_prompt_label_list = []
english_prompt_score_list =[]
english_prompt_label_list = []
for _, row in loaded_df.iterrows():
    danish_prompt = row["danish_predicted_output"]
    danish_prompt = danish_prompt.replace("\n","")
    prediction = predict_language(language_prediction_model,"__label__dan",danish_prompt)
    danish_prompt_score_list.append(round(prediction, 2))
    
    english_prompt = row["english_predicted_output"]
    prediction = predict_language(language_prediction_model,"__label__dan",english_prompt)
    english_prompt_score_list.append(round(prediction,2))

    #danish_prompt_label, danish_prompt_score = language_prediction_model.predict(danish_prompt)
    #danish_prompt_label = danish_prompt_label[0].split("__")[-1]
    #danish_prompt_label_list.append(danish_prompt_label)
    #danish_prompt_score_list.append(round(danish_prompt_score[0],2 ))
    
    #english_prompt = row["english_predicted_output"]
    #english_prompt_label, english_prompt_score =  language_prediction_model.predict(english_prompt)
    #english_prompt_label = english_prompt_label[0].split("__")[-1]
    #english_prompt_label_list.append(english_prompt_label)
    #english_prompt_score_list.append(round(english_prompt_score[0],2))
    


In [20]:
#loaded_df["danish_prompt_label"] = danish_prompt_label_list
loaded_df["danish_prompt_score"] = danish_prompt_score_list
#loaded_df["english_prompt_label"] = english_prompt_label_list
loaded_df["english_prompt_score"] = english_prompt_score_list

In [21]:
loaded_df.to_csv(path + "bible_data_combined.csv", index = False)

In [22]:
print(loaded_df.sample(1).english_predicted_output.iloc[0])

, som døde og to døtre.  The film was released on DVD on 11 April 2012.<|endoftext|>


In [26]:
combined = pd.read_csv(path + "bible_data_combined.csv" )

In [33]:
combined[combined.english_prompt_score > 0.8]

,danish_predicted_output,english_predicted_output,danish_true_label,english_true_label,layer,lambda_amount,bible_verse,danish_prompt_score,english_prompt_score
17,", og fem par af alle de andre som din mand fin...",. 14:30 Og hvis I ikke vil give noget tilover...,"ligeledes af Himmelens Fugle syv Par, Han og H...","Also of the birds of the sky, seven and seven,...",17,2,161,1.00,1.0
27,"undtagen til heste. De skal komme på en mark,...","Det kan du gøre, men det er ikke sikkert, ...","ligeledes af Himmelens Fugle syv Par, Han og H...","Also of the birds of the sky, seven and seven,...",3,5,161,1.00,1.0
28,og af alle ikke levende skabninger af alle dy...,5. 15:23 Han skal ikke spise noget af de and...,"ligeledes af Himmelens Fugle syv Par, Han og H...","Also of the birds of the sky, seven and seven,...",4,5,161,1.00,1.0
31,", fra hver af hans fire horn og fire ørner.<|e...",". of the animals that are not clean, take to t...","ligeledes af Himmelens Fugle syv Par, Han og H...","Also of the birds of the sky, seven and seven,...",7,5,161,1.00,1.0
32,\n27 1. maj 1403 (1789)\n27 1. maj 1403,-> 16:7 Og du skal tage syv Par af hvert ren...,"ligeledes af Himmelens Fugle syv Par, Han og H...","Also of the birds of the sky, seven and seven,...",8,5,161,0.02,1.0
...,...,...,...,...,...,...,...,...,...
54519,"så blev der intet at se, ingen til at hjælpe....",danska: Det er en af de vigtigste årsager ...,"femten Alen stod Vandet over dem, så Bjergene ...","The waters prevailed fifteen cubits upward, an...",15,15,178,1.00,1.0
54520,og under disse syv dage sank alle mennesker i...,"den anden, og til, at, i, 1, 2, 1, 1, 1, 2, 1...","femten Alen stod Vandet over dem, så Bjergene ...","The waters prevailed fifteen cubits upward, an...",16,15,178,1.00,1.0
54521,"der var ikke længere en jordoverflade, kun en...","I skal gaa i deres Dage, og jeg vil væ...","femten Alen stod Vandet over dem, så Bjergene ...","The waters prevailed fifteen cubits upward, an...",17,15,178,1.00,1.0
54522,"og da de var kommet højt op, gav de sig til a...",. der er ved om!<|endoftext|>,"femten Alen stod Vandet over dem, så Bjergene ...","The waters prevailed fifteen cubits upward, an...",18,15,178,1.00,1.0
